In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier


In [2]:
df = pd.read_csv('diabetes.csv')
df

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
x = df.drop('class',axis=1)
y = df['class']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)


In [12]:
scalar = StandardScaler()
x_train_scaled = scalar.fit_transform(x_train)
x_test_scaled = scalar.transform(x_test)
print(x_train_scaled)
print("------------------------------------------------")
print(x_test_scaled)

[[-0.52639686 -1.15139792 -3.75268255 ... -4.13525578 -0.49073479
  -1.03594038]
 [ 1.58804586 -0.27664283  0.68034485 ... -0.48916881  2.41502991
   1.48710085]
 [-0.82846011  0.56687102 -1.2658623  ... -0.42452187  0.54916055
  -0.94893896]
 ...
 [ 1.8901091  -0.62029661  0.89659009 ...  1.76054443  1.981245
   0.44308379]
 [-1.13052335  0.62935353 -3.75268255 ...  1.34680407 -0.78487662
  -0.33992901]
 [-1.13052335  0.12949347  1.43720319 ... -1.22614383 -0.61552223
  -1.03594038]]
------------------------------------------------
[[ 0.68185612 -0.71402038 -0.61712658 ...  0.26073561 -0.11637247
   0.87809089]
 [-0.52639686 -0.27664283  0.30191569 ...  0.48053518 -0.954231
  -1.03594038]
 [-0.52639686 -0.40160784 -0.29275872 ... -0.15300476 -0.9245197
  -1.03594038]
 ...
 [ 1.28598261 -0.80774414  0.13973176 ...  0.62275843  0.04703966
   2.0961108 ]
 [-0.52639686  0.78555979  0.03160914 ... -0.51502758 -0.39268751
  -0.33992901]
 [ 1.28598261 -1.46381046  0.03160914 ...  0.42881763 

In [13]:
xgb = XGBClassifier(use_label_encoder=False,eval_metric = 'logloss',random_state= 42)
param_grid = {
    'n_estimators':[100,150,200,300],
    'learnig_rate':[0.01,0.1,0.15],##gradoant descent learning 
    'max_depth':[2,3,4,5],
    'subsample':[0.8,0,1],
    'colsample_bytree':[0.8,1.0]
}
skf = StratifiedKFold(n_splits = 5,shuffle=True,random_state=42)
grid_search = GridSearchCV(estimator=xgb,
                           param_grid=param_grid,
                           scoring='recall',
                           cv=skf,
                           verbose=1,
                           n_jobs = -1)

    

In [14]:
grid_search.fit(x_train_scaled, y_train)
best_model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation recall:", grid_search.best_score_)
y_pred = best_model.predict(x_test_scaled)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Best parameters: {'colsample_bytree': 1.0, 'learnig_rate': 0.01, 'max_depth': 2, 'n_estimators': 100, 'subsample': 1}
Best cross-validation recall: 0.6673311184939091


C:\Users\Adapa Ved\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:16:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "learnig_rate", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [15]:
print("\n confusion matrix:\n",confusion_matrix(y_test,y_pred))
print("\n clSSIFICATION REPORT:\n",classification_report(y_test,y_pred))


 confusion matrix:
 [[76 23]
 [18 37]]

 clSSIFICATION REPORT:
               precision    recall  f1-score   support

           0       0.81      0.77      0.79        99
           1       0.62      0.67      0.64        55

    accuracy                           0.73       154
   macro avg       0.71      0.72      0.72       154
weighted avg       0.74      0.73      0.74       154



In [17]:
best_model.feature_importances_

array([0.09631848, 0.26798278, 0.05850265, 0.09176092, 0.08672103,
       0.15362413, 0.07190876, 0.17318125], dtype=float32)

In [19]:
features = pd.DataFrame(best_model.feature_importances_,index=df.iloc[:,:-1].columns,columns=['Importances'])
df1=features.sort_values(by = 'Importances')


In [ ]:
import seaborn as sns
sns.varplot(data=df1,x=features.index,y="Importances",hue